In [ ]:
#Manipulacion de datos y funciones matemáticas
import pandas as pd
import numpy as np

#Redes neuronales
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from spacy.lang.es import STOP_WORDS

# Cargar datos
data = pd.read_csv('./Datasets/train.csv')
data_test = pd.read_csv('./Datasets/test.csv')

In [ ]:
# Convertir sentimientos a valores binarios
data['bool_sentimiento'] = (data['sentimiento'] == 'positivo').astype(int)

# Separar datos
x_train, x_test, y_train, y_test = train_test_split(data['review_es'], data['bool_sentimiento'], test_size=0.3, random_state=30)

In [ ]:
# Tokenización y Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

max_length = max(len(seq) for seq in x_train_seq)
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post')



In [17]:
# Creación del modelo
modelo = Sequential()
modelo.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
modelo.add(Bidirectional(LSTM(32, return_sequences=True)))
modelo.add(Bidirectional(LSTM(16)))
modelo.add(Dense(1, activation='sigmoid'))  # Usar 'sigmoid' para clasificación binaria

# Compilación del modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
modelo.fit(x_train_pad, y_train, epochs=1, batch_size=32, validation_data=(x_test_pad, y_test))

# Evaluación del modelo
resultados = modelo.evaluate(x_test_pad, y_test)
print(f"Precisión del modelo: {resultados[1]*100:.2f}%")

# Predicción con el modelo
predicciones = modelo.predict(x_test_pad)


469/469 [==============================] - 103s 219ms/step - loss: 0.4368 - accuracy: 0.8100
Precisión del modelo: 81.00%
469/469 [==============================] - 103s 217ms/step


In [ ]:
""" predicciones = modelo.predict(x_test_pad)

umbral = 0.5
predicciones_binarias = [1 if p > umbral else 0 for p in predicciones]

accuracy = accuracy_score(y_test, predicciones_binarias)
precision = precision_score(y_test, predicciones_binarias)
recall = recall_score(y_test, predicciones_binarias)
f1 = f1_score(y_test, predicciones_binarias)

print("Accuracy:", accuracy)
print("Precisión:", precision)
print("Recall:", recall)
print("F1-Score:", f1) """